##API-Data-Extraction-and-Transformation-for-ERP-Integration

The script is a Python-based data extraction, transformation, and loading (ETL) project. Here is what the script does:

Data Extraction (API calls):

It defines a function get_all_resources_url(url). This function fetches all the data from an API URL provided by a report builder. It fetches all pages of data until there's no more data to fetch. It stores the data in a list, then converts this list to a pandas DataFrame, and returns it. The function fetch_combine_store_data(urls, table_name, key) uses the get_all_resources_url(url) function to fetch data from a list of URLs, combines this data into a single DataFrame, and stores this DataFrame as a CSV file on the specified path. Data Transformation (Cleaning and reshaping the data):

The remove_high_null_columns(df, col_null_threshold=0.05) function removes columns from a DataFrame that have a high percentage of null values based on a specified threshold. It returns a cleaned DataFrame. The function clean_store_data(df, table_name, col_null_threshold=0.05) utilizes the above function to clean a DataFrame, then it stores the cleaned DataFrame as a CSV file on the specified path. Following the cleaning, it performs additional transformations on the DataFrame like splitting column values, removing certain characters, and adjusting leading and trailing spaces. It also deals with duplicates in the data by adding a count number in brackets for duplicated values. Data Loading:

The final DataFrame is saved to a CSV file. As a part of this ETL pipeline, the cleaned and transformed data is prepared for uploading into another system (for example, an ERP system or a database).

The purpose of this script is to pull data from multiple APIs, clean and transform the data as needed, and finally, prepare it for loading into another system or application. This kind of process is common in data analysis and data science projects, where data needs to be moved and transformed between different systems.

###Libraries and Settings

In [ ]:
import requests
import base64
import json
import os
import pandas as pd
from google.colab import drive
from itertools import islice
from ast import literal_eval
import time
drive.mount("/content/drive")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
col_null_threshold=1

Mounted at /content/drive


###Functions

In [ ]:
def get_all_resources_url(url):

    """
    Retrieve all data from an API link generated by a report builder.

    Args:
    url (str): The API URL for fetching data from the report builder.

    Returns:
    df (pd.DataFrame): A DataFrame containing all the data retrieved
                      from the specified API URL.

    Example:
    >>> url = "https://api.example.com/v1/data?pageindex="
    >>> data = get_all_resources_url(url)
    """

    # Initialize an empty list to store the resources
    resources = []

    # Loop through all pages of resources
    page_index = 0
    while True:
        # Get the resources with the specified page index
        response = requests.get(url + str(page_index))

        # Check for successful response
        if response.status_code == 200:
            json_data = json.loads(response.text)
        else:
            print(f'Error fetching data: {response.status_code}')
            break

        # Check if there are no more resources to retrieve
        if json_data == "No records found.":
            break

        # Add the resources to the list
        resources.extend(json_data['data'])

        # Increment the page index for the next iteration
        page_index += 1

    # Create a pandas DataFrame from the list of resources
    df = pd.DataFrame(resources)

    # Return the DataFrame
    return df

def fetch_combine_store_data(urls, table_name, key):

    """
    Fetch data from multiple URLs, combine the data, and store it in a CSV file.

    This function takes a list of URLs and a table name as input. It fetches data from each URL
    using the `get_all_resources_url` function, combines the data into a single DataFrame, and
    saves the combined data to a CSV file with the specified table name.

    Args:
    urls (list): A list of strings, each representing a URL to fetch data from.
    table_name (str): A string representing the name of the table to store the combined data in.
    key: The column name to join on

    Returns:
    pandas.DataFrame: A DataFrame containing the combined data from all the URLs.

    Raises:
    ValueEroor: if key is nt in a data table
    """

    # Initialize an empty DataFrame to store the combined data
    combined_data = get_all_resources_url(urls[0])

    # Loop through the URLs, fetch data from each URL, and concatenate the data
    for url in urls[1:]:
        df = get_all_resources_url(url)
        if key in df.columns:
          combined_data = combined_data.merge(df, on=key)
        else:
          raise ValueError(f"Key does not exist in the data provided by {url}")

    # Define the output file path
    output_file_path = f"/content/drive/My Drive/Projects/Shupper_Brickle/{table_name}_v2.csv"

    # Save the combined data to a CSV file
    combined_data.to_csv(output_file_path, index=False)

    return combined_data

def remove_high_null_columns(df, col_null_threshold=0.05):
    """
    Remove columns with a high percentage of null values from a DataFrame.

    Parameters:
    df (DataFrame): Input DataFrame to be cleaned
    col_null_threshold (float): Threshold for the percentage of null values to remove a column (default is 0.05)
    table_name: table name

    Returns:
    DataFrame: Cleaned DataFrame with columns containing a high percentage of null values removed
    """
    high_null_count = df.isnull().sum() > df.shape[0] * col_null_threshold
    columns_to_retain = list(high_null_count[high_null_count == False].index)
    df_clean = df[columns_to_retain]
    return (df_clean)


    return df_clean
def clean_store_data(df, table_name, col_null_threshold=0.05):

    df_clean = remove_high_null_columns(df, col_null_threshold)
    output_file_path = f"/content/drive/My Drive/Projects/Shupper_Brickle/{table_name}_Clean_v2.csv"
    df_clean.to_csv(output_file_path, index=False)
    return (df_clean)

###Cleaning the Item Data and Storing it as a CSV for ERP Integration

In [ ]:
# List of URLs for the custom reports

Items_urls = [

    ]

Items = fetch_combine_store_data(urls=Items_urls, table_name="Items", key="ID")
# Print the shape of the DataFrame
print(f"Items Shape = {Items.shape}")

Items_Clean = clean_store_data(Items, "Items", col_null_threshold)
Items_Clean.head(2)

# Create a working copy of the relevant columns from the cleaned items DataFrame
df = Items_Clean[["ID", "ItemName", "ItemNumber"]].copy()

# Split "ItemName" at the first space and assign the first part (item number) to "NewItemNumber"
df["NewItemNumber"] = df.ItemName.apply(lambda x : x.split(' ',1)[0])

# If the last character of "NewItemNumber" is "-" or ",", remove it
df["NewItemNumber"] = df.NewItemNumber.apply(lambda x : x[:-1] if x[-1] in  ["-", ","] else x)

# Split "ItemName" again and assign the second part (item name) to "NewItemName".
# If there's no space in the string, retain the whole string.
df["NewItemName"] = df.ItemName.apply(lambda x : x.split(' ', 1)[1] if ' ' in x else x)

# Remove leading and trailing spaces from "NewItemName" and "NewItemNumber"
df["NewItemName"] = df["NewItemName"].str.strip()
df["NewItemNumber"] = df["NewItemNumber"].str.strip()

# Re-order columns for clarity
df = df[["ID", "ItemName", "ItemNumber", "NewItemNumber", "NewItemName"]]

# Identify and display duplicates based on "NewItemNumber"
ids = df[df["NewItemNumber"].duplicated(keep=False)].sort_values("NewItemNumber")["ID"]
#print("Duplicated rows:")
#display(df[df["NewItemNumber"].duplicated(keep=False)].sort_values("NewItemNumber"))

# Create a new DataFrame to fix duplicates
df3 = df.copy()
df3['count'] = df3.groupby('NewItemNumber').cumcount() + 1
df3['NewItemNumber'] = df3.apply(lambda row: row['NewItemNumber'] + '(' + str(row['count'])+ ')' if row['count'] != 1 else row['NewItemNumber'], axis=1)

# Discard the temporary 'count' column
df3 = df3.drop('count', axis=1)

# Display the fixed duplicated rows for verification
#print("Fixed duplicated rows:")
#display(df3[df3["ID"].isin(ids)].sort_values("NewItemNumber"))

# Show a sample from the final DataFrame
#display(df3.head(10))

# Save the final DataFrame as a CSV file
df3.to_csv("/content/drive/MyDrive/Projects/Shupper_Brickle/Project 15 Items Data/items-data-v1.csv", index= False)

#Upload the df3 to striven

Items Shape = (2945, 32)
